In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LSTM, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers, metrics, regularizers
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
train_dataX = np.load('augmentedData_Combined.npy')
test_dataX = np.load('test_data.npy')
train_datay = pd.read_csv('AugmentedCombined_labels.csv')

In [3]:
#pre-processing data
train_datay = train_datay.drop('Id',1)
train_dataX = train_dataX.reshape(-1,210,210)
test_dataX = test_dataX.reshape(-1,210,210)
Y_train = np_utils.to_categorical(train_datay,2)

#splitting data for validation
X_train, X_test, y_train, y_test = train_test_split(train_dataX,Y_train, test_size=0.08, shuffle=True)

#deleting unwanted variables
del train_dataX, train_datay

In [6]:
#del model
model = Sequential()
model.add(Conv1D(16 ,kernel_size=3 ,activation='relu' ,input_shape=(210,210)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(32, kernel_size=3,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(64, kernel_size=3,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
#model.add(SimpleRNN(128))
model.add(LSTM(200))
model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 208, 16)           10096     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 104, 16)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 104, 16)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 102, 32)           1568      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 51, 32)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 51, 32)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 49, 64)            6208      
__________

In [7]:
# compile model
sgd=optimizers.SGD(lr=0.01) 
adam = optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train,
          batch_size=250, nb_epoch=20,verbose=1,
          validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.


d:\programs\python installed\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 22074 samples, validate on 1920 samples
Epoch 1/20
22074/22074 [==============================] - 24s 1ms/step - loss: 0.2196 - acc: 0.9057 - val_loss: 0.1648 - val_acc: 0.9245
Epoch 2/20
22074/22074 [==============================] - 12s 562us/step - loss: 0.1762 - acc: 0.9198 - val_loss: 0.1343 - val_acc: 0.9469
Epoch 3/20
22074/22074 [==============================] - 12s 536us/step - loss: 0.1474 - acc: 0.9370 - val_loss: 0.1235 - val_acc: 0.9443
Epoch 4/20
22074/22074 [==============================] - 11s 501us/step - loss: 0.1217 - acc: 0.9487 - val_loss: 0.1183 - val_acc: 0.9552
Epoch 5/20
22074/22074 [==============================] - 11s 505us/step - loss: 0.1196 - acc: 0.9501 - val_loss: 0.1082 - val_acc: 0.9552
Epoch 6/20
22074/22074 [==============================] - 11s 492us/step - loss: 0.1083 - acc: 0.9551 - val_loss: 0.1004 - val_acc: 0.9589
Epoch 7/20
22074/22074 [==============================] - 10s 448us/step - loss: 0.1054 - acc: 0.9566 - val_loss: 0.103

In [9]:
from sklearn.metrics import roc_auc_score

Y_test_predClass = model.predict_proba(X_test)

roc = roc_auc_score(y_test, Y_test_predClass)
print("ROC:" + str(round(roc,4)))

ROC:0.9882


In [11]:
Y_test_pred = model.predict_proba(test_dataX)
print(Y_test_pred)

[[9.9966538e-01 3.3468613e-04]
 [9.9968231e-01 3.1772556e-04]
 [9.9952722e-01 4.7282601e-04]
 ...
 [9.9995852e-01 4.1443225e-05]
 [9.9961770e-01 3.8228699e-04]
 [1.4270164e-02 9.8572981e-01]]


In [12]:
pd.DataFrame(Y_test_pred[:,0]).to_csv('submission57.csv', header=["Label"], index_label="Id")